# The Physics of Sound, Part II

[return to main page](index.ipynb)

## Preparations

If it's not installed already, you can install the SFS module with:

    python3 -m pip install sfs --user

If you have only Python 3 installed on your system, you may have to use `python` instead of `python3`.

Afterwards, you should re-start any running IPython kernels using the tap named "Kernel"

Once installed, we can import it into our Python session:


In [ ]:
import sfs

And some other stuff:

In [ ]:
# remove "inline" to get a separate plotting window:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.umath_tests import inner1d

A grid for plotting:

In [ ]:
grid = sfs.util.xyz_grid([-2, 2], [-2, 2], 0, spacing=0.01)

Have a look at the [documentation](http://sfs.readthedocs.org/en/latest/#sfs.util.xyz_grid) to find out what the function parameters mean.

*Exercise:* What does the third argument mean in our case?
How many dimensions does our grid have?

## The Helmholtz Integral Equation

In [ ]:
xs = 0,2,0  # position (metre)
f = 1000  # time-frequency (Hz)
omega = 2 * np.pi * f  # angular frequency (rad/s)

x0, n0, a0 = sfs.array.circular(100, 1)  # secondary source on the circular boundary
x0_grid = np.array([x0[:,0],x0[:,1],x0[:,2]])  # convert to grid

p = sfs.mono.source.point(omega, xs, None, x0_grid)  # sound pressure at boundary 
v = sfs.mono.source.point_velocity(omega, xs, None, x0_grid)*(-1j)*sfs.defs.rho0*omega # pressure gradient at boundary
vx, vy, vz = sfs.util.as_xyz_components(v)
v = np.array([vx,vy,vz]).T

# directional gradient ("pressure gradient along n0")
v_n0 = np.einsum('ij,ij->i', v, n0)

# single layer potential (sound field generate by secondary monopole source)
p_single = sfs.mono.synthesized.generic(omega, x0, n0, -v_n0 * a0 , grid, source=sfs.mono.source.point)
# double layer potential (sound field generated by secondary dipole source)
p_double = sfs.mono.synthesized.generic(omega, x0, n0, p * a0 , grid, source=sfs.mono.source.point_dipole)

sfs.plot.soundfield(4*np.pi*(p_single + p_double), grid);
sfs.plot.loudspeaker_2d(x0,n0)